In [21]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [22]:
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
def dataset():
    train_dataset = torchvision.datasets.MNIST(root='data/train', train=True, transform=transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()]), download=True)
    test_dataset = torchvision.datasets.MNIST(root='data/train', train=False, transform=transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()]), download=True)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
    return train_loader, test_loader

In [24]:
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [25]:
model = LeNet5(num_classes).to(device)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [26]:
def train():

    print(f"Running on {device}")
    train_loader, test_loader = dataset()
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):  
            images = images.to(device)
            labels = labels.to(device)
            
            #Forward pass
            outputs = model(images)
            loss = cost(outputs, labels)
                
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
                    
            if (i+1) % 400 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                            .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


In [27]:
train()

Running on cuda
Epoch [1/10], Step [400/938], Loss: 0.1549
Epoch [1/10], Step [800/938], Loss: 0.0615
Epoch [2/10], Step [400/938], Loss: 0.0197
Epoch [2/10], Step [800/938], Loss: 0.0286
Epoch [3/10], Step [400/938], Loss: 0.0364
Epoch [3/10], Step [800/938], Loss: 0.1566
Epoch [4/10], Step [400/938], Loss: 0.2407
Epoch [4/10], Step [800/938], Loss: 0.0366
Epoch [5/10], Step [400/938], Loss: 0.0028
Epoch [5/10], Step [800/938], Loss: 0.0080
Epoch [6/10], Step [400/938], Loss: 0.0014
Epoch [6/10], Step [800/938], Loss: 0.0038
Epoch [7/10], Step [400/938], Loss: 0.0015
Epoch [7/10], Step [800/938], Loss: 0.0025
Epoch [8/10], Step [400/938], Loss: 0.0242
Epoch [8/10], Step [800/938], Loss: 0.0007
Epoch [9/10], Step [400/938], Loss: 0.0032
Epoch [9/10], Step [800/938], Loss: 0.0166
Epoch [10/10], Step [400/938], Loss: 0.0007
Epoch [10/10], Step [800/938], Loss: 0.0007


In [28]:
with torch.no_grad():
    train_loader, test_loader = dataset()
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 99.14 %
